In [8]:
import pandas as pd
import numpy as np
import pickle

In [14]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics: 
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [15]:
calendar_file = "D:\\\m5-forecasting-accuracy\\calendar.csv"
sales_train_validation_file = "D:\\\m5-forecasting-accuracy\\sales_train_validation.csv"
sell_prices_file = "D:\\\m5-forecasting-accuracy\\sell_prices.csv"


calendar_ = pd.read_csv(calendar_file, delimiter=",")
sales_train_validation_ = pd.read_csv(sales_train_validation_file, delimiter=",")
sell_prices_ = pd.read_csv(sell_prices_file, delimiter=",")

In [16]:
calendar_ = reduce_mem_usage(calendar_)

Mem. usage decreased to  0.12 Mb (41.9% reduction)


In [18]:
sell_prices_ = reduce_mem_usage(sell_prices_)

Mem. usage decreased to 130.48 Mb (37.5% reduction)


In [19]:
print(calendar_.shape)
calendar_.head()

(1969, 14)


,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


In [20]:
print(sales_train_validation_.shape)
sales_train_validation_.head()

(30490, 1919)


,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4


In [21]:
print(sell_prices_.shape)
sell_prices_.head()

(6841121, 4)


,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.578125
1,CA_1,HOBBIES_1_001,11326,9.578125
2,CA_1,HOBBIES_1_001,11327,8.257812
3,CA_1,HOBBIES_1_001,11328,8.257812
4,CA_1,HOBBIES_1_001,11329,8.257812


## Prepare Training Data

In [22]:
column_names = list(sales_train_validation_.columns)
train_ = pd.melt(sales_train_validation_, id_vars=column_names[0:6],value_vars=column_names[6:],var_name='d', value_name='sales')

In [23]:
print(train_.shape)
train_.head()

(58327370, 8)


,id,item_id,dept_id,cat_id,store_id,state_id,d,sales
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0


In [24]:
train_ = reduce_mem_usage(train_)

Mem. usage decreased to 3226.27 Mb (9.4% reduction)


#### Merge with calendar

In [25]:
train = pd.merge(train_, calendar_, on='d')

In [26]:
train_data = train.drop(['event_name_1','event_type_1','event_name_2','event_type_2','snap_CA','snap_TX','snap_WI'], axis=1, inplace=True)

In [76]:
train.shape

(58327370, 14)

In [29]:
train.columns

Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd',
       'sales', 'date', 'wm_yr_wk', 'weekday', 'wday', 'month', 'year'],
      dtype='object')

#### Merge with sell_prices

In [30]:
train_data = train.merge(sell_prices_, on=["store_id","item_id","wm_yr_wk"])

In [31]:
train_data.shape

(46027957, 15)

In [32]:
train.head()
train.shape

(58327370, 14)

### Generate simple features

In [39]:
def lag_(df,lag_val):
   
    # sales: lag    
    col = f"lag_{lag_val}"
    df.loc[:,col] = df.groupby(['id'])['sales'].transform(lambda x: x.shift(lag_val))
    
    return df

In [57]:
def rolling_mean_(df,lag_val,days):
   
    # sales: rolling_mean   
    col = f"mov_avg_{days}_{lag_val}"
    df.loc[:,col] = df.loc[:,f"lag_{lag_val}"].transform(lambda x: x.rolling(days).mean())
    
    return df

In [68]:
def simple_features_monthly(df):
    
    # monthly mean, std minimum/max sales
    months = set(df['month'])
    
    for month in months:        
        df.loc[df['month'] == month,'montly_sales_mean'] = df.loc[df['month'] == month].groupby('id')['sales'].transform(lambda x: x.mean())
        df.loc[df['month'] == month,'montly_sales_std'] = df.loc[df['month'] == month].groupby('id')['sales'].transform(lambda x: x.std())
        df.loc[df['month'] == month,'monthly_sales_max'] = df.loc[df['month'] == month].groupby('id')['sales'].transform(lambda x: x.max())
        df.loc[df['month'] == month,'monthly_sales_min'] = df.loc[df['month'] == month].groupby('id')['sales'].transform(lambda x: x.min())
        print("done")
    
    return df

In [69]:
def simple_features_weekly(df):
    
    # weekly mean, std minimum/max sales, max-min 
    weekdays = set(df['weekday'])
    
    for day in weekdays:
        
        df.loc[df['weekday'] == day,'weekly_sales_mean'] = df.loc[df['weekday'] == day].groupby('id')['sales'].transform(lambda x: x.mean())
        df.loc[df['weekday'] == day,'weekly_sales_std'] = df.loc[df['weekday'] == day].groupby('id')['sales'].transform(lambda x: x.std())
        df.loc[df['weekday'] == day,'weekly_sales_max'] = df.loc[df['weekday'] == day].groupby('id')['sales'].transform(lambda x: x.max())
        df.loc[df['weekday'] == day,'weekly_sales_min'] = df.loc[df['weekday'] == day].groupby('id')['sales'].transform(lambda x: x.min())
        print("done")
        
    return df

In [37]:
train_data = lag_(train_data,7)

In [42]:
train_data.to_pickle("./lag_7.pkl")

In [44]:
train_data = lag_(train_data,28)

In [46]:
train_data.to_pickle("D://LSDA_pickles//lag_28.pkl")

In [50]:
train_data = rolling_mean_(train_data,7,7)

In [52]:
train_data.columns

Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd',
       'sales', 'date', 'wm_yr_wk', 'weekday', 'wday', 'month', 'year',
       'sell_price', 'lag_7', 'lag_28', 'mov_avg_7_7'],
      dtype='object')

In [53]:
train_data = rolling_mean_(train_data,7,28)

In [54]:
train_data = rolling_mean_(train_data,28,7)

In [55]:
train_data = rolling_mean_(train_data,28,28)

In [56]:
train_data.to_pickle("D://LSDA_pickles//lag_rolling_mean_features.pkl")

In [70]:
train_data = simple_features_monthly(train_data)

done
done
done
done
done
done
done
done
done
done
done
done


In [71]:
train_data.to_pickle("D://LSDA_pickles//lag_rolling_mean_monthly_features.pkl")

In [72]:
train_data = simple_features_weekly(train_data)

done
done
done
done
done
done
done


In [73]:
train_data.to_pickle("D://LSDA_pickles//lag_rolling_mean_monthly_weekly_features.pkl")

In [74]:
train_data.shape

(46027957, 29)

In [77]:
train_data.columns

Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd',
       'sales', 'date', 'wm_yr_wk', 'weekday', 'wday', 'month', 'year',
       'sell_price', 'lag_7', 'lag_28', 'mov_avg_7_7', 'mov_avg_28_7',
       'mov_avg_7_28', 'mov_avg_28_28', 'montly_sales_mean',
       'montly_sales_std', 'monthly_sales_max', 'monthly_sales_min',
       'weekly_sales_mean', 'weekly_sales_std', 'weekly_sales_max',
       'weekly_sales_min'],
      dtype='object')

In [79]:
58327370-46027957

12299413

In [82]:
train_data.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,...,mov_avg_7_28,mov_avg_28_28,montly_sales_mean,montly_sales_std,monthly_sales_max,monthly_sales_min,weekly_sales_mean,weekly_sales_std,weekly_sales_max,weekly_sales_min
0,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_1,12,2011-01-29,11101,...,NaN,NaN,8.500000,10.676929,75.0,0.0,8.613139,8.786226,47.0,0.0
1,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_2,15,2011-01-30,11101,...,NaN,NaN,8.500000,10.676929,75.0,0.0,6.394161,7.405431,45.0,0.0
2,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_3,0,2011-01-31,11101,...,NaN,NaN,8.500000,10.676929,75.0,0.0,7.058608,9.588033,54.0,0.0
3,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_4,0,2011-02-01,11101,...,NaN,NaN,9.170588,11.581472,91.0,0.0,6.542125,8.202350,50.0,0.0
4,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_5,0,2011-02-02,11101,...,NaN,NaN,9.170588,11.581472,91.0,0.0,7.560440,9.495985,48.0,0.0


In [83]:
grouped_ = train_data.groupby('id')

In [84]:
item_df = (grouped_.get_group('FOODS_3_586_CA_1_validation'))

In [93]:
item_df[100:140]

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,...,mov_avg_7_28,mov_avg_28_28,montly_sales_mean,montly_sales_std,monthly_sales_max,monthly_sales_min,weekly_sales_mean,weekly_sales_std,weekly_sales_max,weekly_sales_min
1328049,FOODS_3_586_CA_1_validation,FOODS_3_586,FOODS_3,FOODS,CA_1,CA,d_101,36,2011-05-09,11115,...,23.571429,6.964286,45.567742,14.898365,87.0,17.0,40.673993,11.415931,76.0,20.0
1328050,FOODS_3_586_CA_1_validation,FOODS_3_586,FOODS_3,FOODS,CA_1,CA,d_102,19,2011-05-10,11115,...,30.142857,8.607143,45.567742,14.898365,87.0,17.0,36.153846,10.068066,66.0,0.0
1328051,FOODS_3_586_CA_1_validation,FOODS_3_586,FOODS_3,FOODS,CA_1,CA,d_103,28,2011-05-11,11115,...,32.857143,9.321429,45.567742,14.898365,87.0,17.0,36.692308,10.628097,72.0,0.0
1328052,FOODS_3_586_CA_1_validation,FOODS_3_586,FOODS_3,FOODS,CA_1,CA,d_104,34,2011-05-12,11115,...,36.571429,10.214286,45.567742,14.898365,87.0,17.0,38.838828,11.377369,85.0,0.0
1328053,FOODS_3_586_CA_1_validation,FOODS_3_586,FOODS_3,FOODS,CA_1,CA,d_105,62,2011-05-13,11115,...,41.857143,11.464286,45.567742,14.898365,87.0,17.0,45.677656,14.006516,133.0,0.0
1431857,FOODS_3_586_CA_1_validation,FOODS_3_586,FOODS_3,FOODS,CA_1,CA,d_106,51,2011-05-14,11116,...,9.142857,2.535714,45.567742,14.898365,87.0,17.0,58.076642,13.507750,111.0,30.0
1431858,FOODS_3_586_CA_1_validation,FOODS_3_586,FOODS_3,FOODS,CA_1,CA,d_107,68,2011-05-15,11116,...,18.857143,5.000000,45.567742,14.898365,87.0,17.0,64.649635,16.581949,114.0,0.0
1431859,FOODS_3_586_CA_1_validation,FOODS_3_586,FOODS_3,FOODS,CA_1,CA,d_108,33,2011-05-16,11116,...,24.714286,6.500000,45.567742,14.898365,87.0,17.0,40.673993,11.415931,76.0,20.0
1431860,FOODS_3_586_CA_1_validation,FOODS_3_586,FOODS_3,FOODS,CA_1,CA,d_109,29,2011-05-17,11116,...,28.714286,7.464286,45.567742,14.898365,87.0,17.0,36.153846,10.068066,66.0,0.0
1431861,FOODS_3_586_CA_1_validation,FOODS_3_586,FOODS_3,FOODS,CA_1,CA,d_110,26,2011-05-18,11116,...,33.857143,8.750000,45.567742,14.898365,87.0,17.0,36.692308,10.628097,72.0,0.0
